In [3]:
df = spark.read.format("csv").option("header","true").load("Files/FraudData/Account_Info.csv")
# df now is a Spark DataFrame containing CSV data from "Files/FraudData/Account_Info.csv".
display(df)

StatementMeta(, 950af6b9-c8fd-4684-b6f9-895e19931fbb, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5f972b5b-bca4-4635-8eef-ad0400a0c078)

In [2]:
df.write.mode("overwrite").saveAsTable("Account_Info")

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 4, Finished, Available)

In [1]:
df = spark.read.format("csv").option("header","true").load("Files/FraudData/Fraud_Transactions.csv")
# df now is a Spark DataFrame containing CSV data from "Files/FraudData/Fraud_Transactions.csv".
df.write.mode("overwrite").saveAsTable("Fraud_Transactions")

StatementMeta(, 950af6b9-c8fd-4684-b6f9-895e19931fbb, 3, Finished, Available)

In [4]:
df = spark.read.format("csv").option("header","true").load("Files/FraudData/Untagged_Transactions.csv")
# df now is a Spark DataFrame containing CSV data from "Files/FraudData/Untagged_Transactions.csv".
df.write.mode("overwrite").saveAsTable("Untagged_Transactions")

StatementMeta(, 950af6b9-c8fd-4684-b6f9-895e19931fbb, 6, Finished, Available)

In [40]:
%%sql
CREATE TABLE lakehousebronze.untagged_formatted AS
SELECT
a.*,
to_timestamp(
concat(a.transactionDate,' ',
CASE 
    WHEN len(a.transactionTime ) = 5 THEN concat('0',a.transactionTime)
    WHEN len(a.transactionTime ) = 4 THEN concat('00',a.transactionTime)
    WHEN len(a.transactionTime ) = 3 THEN concat('000',a.transactionTime)
    WHEN len(a.transactionTime ) = 2 THEN concat('0000',a.transactionTime)
    WHEN len(a.transactionTime ) = 1 THEN concat('00000',a.transactionTime)
    ELSE a.transactionTime
END
),"yyyyMMdd HHmmss") AS transactionDateTime
FROM lakehousebronze.untagged_transactions a

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 42, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
DROP TABLE lakehousebronze.ACCT_INFO_MERGED;
CREATE TABLE lakehousebronze.ACCT_INFO_MERGED AS
select * from 
(
select a.*,
        t.accountOwnerName as l_accountOwnerName,
        t.accountAddress l_accountAddress,
        t.accountPostalCode l_accountPostalCode,
        t.accountCity l_accountCity,
        t.accountState l_accountState,
        t.accountCountry l_accountCountry,
        t.accountOpenDate l_accountOpenDate,
        t.accountAge l_accountAge,
        t.isUserRegistered l_isUserRegistered,
        t.paymentInstrumentAgeInAccount l_paymentInstrumentAgeInAccount,
        t.numPaymentRejects1dPerUser l_numPaymentRejects1dPerUsert,
        Rank() over (PARTITION BY a.accountID,a.transactionID
                        ORDER BY rnk asc) as frnk
  from 
  untagged_formatted  a
  left join
  (
        SELECT 
        a.*,
        to_timestamp(
        concat(a.transactionDate,' ',
        CASE 
            WHEN len(a.transactionTime ) = 5 THEN concat('0',a.transactionTime)
            WHEN len(a.transactionTime ) = 4 THEN concat('00',a.transactionTime)
            WHEN len(a.transactionTime ) = 3 THEN concat('000',a.transactionTime)
            WHEN len(a.transactionTime ) = 2 THEN concat('0000',a.transactionTime)
            WHEN len(a.transactionTime ) = 1 THEN concat('00000',a.transactionTime)
            ELSE a.transactionTime
        END
        ),"yyyyMMdd HHmmss") AS recordDateTime,
        Rank() over (PARTITION BY a.accountID 
        ORDER BY         
        to_timestamp(
        concat(a.transactionDate,' ',
        CASE 
            WHEN len(a.transactionTime ) = 5 THEN concat('0',a.transactionTime)
            WHEN len(a.transactionTime ) = 4 THEN concat('00',a.transactionTime)
            WHEN len(a.transactionTime ) = 3 THEN concat('000',a.transactionTime)
            WHEN len(a.transactionTime ) = 2 THEN concat('0000',a.transactionTime)
            WHEN len(a.transactionTime ) = 1 THEN concat('00000',a.transactionTime)
            ELSE a.transactionTime
        END
        ),"yyyyMMdd HHmmss") desc) as rnk
        from
        lakehousebronze.account_info as a
        
        ) t

 ON
        t.accountID = a.accountID and 
        t.recordDateTime <= a.transactionDateTime
)
where frnk = 1

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
select * from lakehousebronze.ACCT_INFO_MERGED limit 20

StatementMeta(, d7c2a9ad-6dfc-4799-804b-5cd0bdac7cdd, 15, Finished, Available)

<Spark SQL result set with 20 rows and 53 fields>

In [39]:
%%sql
CREATE TABLE lakehousebronze.fraud_formatted AS
SELECT
a.*,
to_timestamp(
concat(a.transactionDate,' ',
CASE 
    WHEN len(a.transactionTime ) = 5 THEN concat('0',a.transactionTime)
    WHEN len(a.transactionTime ) = 4 THEN concat('00',a.transactionTime)
    WHEN len(a.transactionTime ) = 3 THEN concat('000',a.transactionTime)
    WHEN len(a.transactionTime ) = 2 THEN concat('0000',a.transactionTime)
    WHEN len(a.transactionTime ) = 1 THEN concat('00000',a.transactionTime)
    ELSE a.transactionTime
END
),"yyyyMMdd HHmmss") AS transactionDateTime
FROM lakehousebronze.fraud_transactions a

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 41, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
DROP TABLE lakehousebronze.tagged_f;
CREATE TABLE lakehousebronze.tagged_f AS
select 
distinct
t1.*,
  t1.transactionDateTime as tDT,
  t2.startDateNTime as sDT,
  t2.endDateNTime as eDT
 from
lakehousebronze.untagged_formatted t1
LEFT OUTER JOIN
(select 
accountID, 
min(transactionDateTime) as startDateNTime,  
max(transactionDateTime) as endDateNTime
from
lakehousebronze.fraud_formatted ff
group by accountID) t2
ON t1.accountID = t2.accountID

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [19]:
%%sql
select label,count(*) from lakehousebronze.tagged_processed group by 1 

StatementMeta(, d7c2a9ad-6dfc-4799-804b-5cd0bdac7cdd, 24, Finished, Available)

<Spark SQL result set with 2 rows and 2 fields>

In [17]:
%%sql
DROP TABLE lakehousebronze.tagged_processed;
CREATE TABLE lakehousebronze.tagged_processed AS
select
distinct
t.accountID,
t.transactionID,
t.transactionDateTime,
case 
 when (sDT is not null and tDT >= sDT and tDT <= eDT) then 1
 when (sDT is not null and tDT < sDT) then 2
 when (sDT is not null and tDT > eDT) then 2
 when sDT is null then 0
end as label,
coalesce(t.isProxyIP, '0') as isProxyIP, 
coalesce(t.paymentInstrumentType, '0') as paymentInstrumentType,
coalesce(t.cardType, '0') as cardType,
coalesce(t.paymentBillingAddress, '0') as paymentBillingAddress,
coalesce(t.paymentBillingPostalCode, '0') as paymentBillingPostalCode,
coalesce(t.paymentBillingCountryCode, '0') as paymentBillingCountryCode,
coalesce(t.paymentBillingName, '0') as paymentBillingName,
coalesce(a.l_accountAddress, '0') as accountAddress,
coalesce(a.l_accountPostalCode, '0') as accountPostalCode,
coalesce(a.l_accountCountry, '0') as accountCountry,
coalesce(a.l_accountOwnerName, '0') as accountOwnerName,
coalesce(t.shippingAddress, '0') as shippingAddress,
coalesce(t.transactionCurrencyCode, '0') as transactionCurrencyCode,
coalesce(t.localHour,'-99') as localHour,
coalesce(t.ipState, '0') as ipState,
coalesce(t.ipPostCode, '0') as ipPostCode,
coalesce(t.ipCountryCode, '0') as ipCountryCode,
coalesce(t.browserLanguage, '0') as browserLanguage,
coalesce(t.paymentBillingState, '0') as paymentBillingState,
coalesce(a.l_accountState, '0') as accountState,
COALESCE(cast(t.transactionAmountUSD as float),0) as transactionAmountUSD,
COALESCE(cast(t.digitalItemCount as float),0) as digitalItemCount,
COALESCE(cast(t.physicalItemCount as float),0) as physicalItemCount,
COALESCE(cast(a.l_accountAge as float),0) as accountAge,
COALESCE(cast(a.l_paymentInstrumentAgeInAccount as float),0) as paymentInstrumentAgeInAccount,
COALESCE(cast(a.l_numPaymentRejects1dPerUsert as float),0) as numPaymentRejects1dPerUser,
case when a.l_isUserRegistered like '%[0-9]%' then '0' else a.l_isUserRegistered end as isUserRegistered
from tagged_f t
left outer join acct_info_merged a
on 
t.accountID = a.accountID
AND t.transactionID = a.transactionID
where cast(t.transactionAmountUSD as float) >= 0 and   
      (case when t.transactionDateTime is null then 1 else 0 end) = 0 and
	case 
            when (sDT is not null and tDT >= sDT and tDT <= eDT) then 1
            when (sDT is not null and tDT < sDT) then 2
            when (sDT is not null and tDT > eDT) then 2
            when sDT is null then 0
      end < 2 

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [83]:
%%sql
CREATE TABLE Risk_LocalHour AS 
select 
localHour , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct localHour ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by localHour ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 85, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [85]:
%%sql
CREATE TABLE Risk_TransactionCurrencyCode AS 
select 
transactionCurrencyCode , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct transactionCurrencyCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by transactionCurrencyCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 87, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [86]:
%%sql
CREATE TABLE Risk_ipState AS 
select 
ipState , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct ipState ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by ipState ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 88, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [87]:
%%sql
CREATE TABLE Risk_ipPostCode AS 
select 
ipPostCode , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct ipPostCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by ipPostCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 89, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [88]:
%%sql
CREATE TABLE Risk_ipCountryCode AS 
select 
ipCountryCode , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct ipCountryCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by ipCountryCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 90, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [89]:
%%sql
CREATE TABLE Risk_browserLanguage AS 
select 
browserLanguage , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct browserLanguage ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by browserLanguage ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 91, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [90]:
%%sql
CREATE TABLE Risk_paymentBillingPostalCode AS 
select 
paymentBillingPostalCode , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct paymentBillingPostalCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by paymentBillingPostalCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 92, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [91]:
%%sql
CREATE TABLE Risk_paymentBillingState AS 
select 
paymentBillingState , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct paymentBillingState ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by paymentBillingState ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 93, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [92]:
%%sql
CREATE TABLE Risk_paymentBillingCountryCode AS 
select 
paymentBillingCountryCode , 
log(odds/(1-odds)) as risk 
       from 
       (select distinct paymentBillingCountryCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by paymentBillingCountryCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 94, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [93]:
%%sql
CREATE TABLE Risk_accountPostalCode
 AS 
select 
accountPostalCode, 
log(odds/(1-odds)) as risk 
       from 
       (select distinct accountPostalCode ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by accountPostalCode ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 95, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [94]:
%%sql
CREATE TABLE Risk_accountState
 AS 
select 
accountState, 
log(odds/(1-odds)) as risk 
       from 
       (select distinct accountState ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by accountState ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 96, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [95]:
%%sql
CREATE TABLE Risk_accountCountry
 AS 
select 
accountCountry, 
log(odds/(1-odds)) as risk 
       from 
       (select distinct accountCountry ,cast((sum(label)+10) as float)/cast((sum(label)+sum(1-label)+100) as float) as odds 
			from lakehousebronze.tagged_processed 
            group by accountCountry ) temp

StatementMeta(, 25670b64-b13d-4aa8-a7c4-d8f569ddd778, 97, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [17]:
%%sql
DROP TABLE fraud_final_other_txs_30;
CREATE TABLE fraud_final_other_txs_30 AS
SELECT 
t1.accountid ,
t1.transactionID,
t1.TransactionDateTime,
SUM(t1.TransactionAmountUSD) AS sumPurchaseAmount30dPerUser,
SUM(1) AS sumPurchaseCount30dPerUser
FROM untagged_formatted t1
JOIN untagged_formatted t2
ON t1.accountid = t2.accountid
AND ABS(DATEDIFF(day, t1.transactionDateTime, t2.transactionDateTime)) <= 30
AND t1.transactionID != t2.transactionID
AND t1.TransactionDateTime != t2.TransactionDateTime
AND t1.TransactionAmountUSD != t2.TransactionAmountUSD
GROUP BY t1.accountid ,t1.transactionID,t1.TransactionDateTime,t1.TransactionAmountUSD

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [20]:
%%sql
DROP TABLE fraud_final_other_txs_1;
CREATE TABLE fraud_final_other_txs_1 AS
SELECT 
t1.accountid ,
t1.transactionID,
t1.TransactionDateTime,
SUM(t1.TransactionAmountUSD) AS sumPurchaseAmount1dPerUser,
SUM(1) AS sumPurchaseCount1dPerUser
FROM untagged_formatted t1
JOIN untagged_formatted t2
ON t1.accountid = t2.accountid
AND ABS(DATEDIFF(day, t1.transactionDateTime, t2.transactionDateTime)) <= 1
AND t1.transactionID != t2.transactionID
AND t1.TransactionDateTime != t2.TransactionDateTime
AND t1.TransactionAmountUSD != t2.TransactionAmountUSD
GROUP BY t1.accountid ,t1.transactionID,t1.TransactionDateTime,t1.TransactionAmountUSD

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
%%sql
DROP TABLE fraud_final;
CREATE TABLE fraud_final as
SELECT
t1.*,
COALESCE(t2.sumPurchaseCount1dPerUser,0) sumPurchaseCount1dPerUser,
COALESCE(t2.sumPurchaseAmount1dPerUser,0) sumPurchaseAmount1dPerUser,
COALESCE(t2.sumPurchaseCount30dPerUser,0) sumPurchaseCount30dPerUser,
COALESCE(t2.sumPurchaseAmount30dPerUser,0) sumPurchaseAmount30dPerUser
from
(
select 
t.accountID,
t.transactionID,
t.transactionDateTime,
t.label,
EXTRACT(dayofweek from t.transactionDateTime ) as txDayOfWeek,
EXTRACT(quarter from t.transactionDateTime ) as txQuarter,
t.transactionAmountUSD,
t.digitalItemCount,
t.physicalItemCount,
t.isProxyIP,
t.paymentInstrumentType,
t.cardType,
t.isUserRegistered,
t.accountAge,
t.paymentInstrumentAgeInAccount,
t.numPaymentRejects1dPerUser,
case when t.transactionAmountUSD > 150 then '1' else '0' end as isHighAmount,
case when t.paymentBillingAddress = t.accountAddress then '0' else '1' end as acctBillingAddressMismatchFlag,
case when t.paymentBillingPostalCode = t.accountPostalCode then '0' else '1' end as acctBillingPostalCodeMismatchFlag,
case when t.paymentBillingCountryCode = t.accountCountry then '0' else '1' end as acctBillingCountryMismatchFlag,
case when t.paymentBillingName = t.accountOwnerName then '0' else '1' end as acctBillingNameMismatchFlag,
case when t.shippingAddress = t.accountAddress then '0' else '1' end as acctShippingAddressMismatchFlag,
case when t.shippingAddress = t.paymentBillingAddress then '0' else '1' end as shippingBillingAddressMismatchFlag,
COALESCE(ac.risk,0) as accountCountryRisk,
COALESCE(apc.risk,0) as accountPostalCodeRisk,
COALESCE(actst.risk,0) as accountStateRisk,
COALESCE(bl.risk,0) as browserLanguageRisk,
COALESCE(ic.risk,0) as ipCountryCodeRisk,
COALESCE(ipc.risk,0) as ipPostCodeRisk,
COALESCE(ips.risk,0) as ipStateRisk,
COALESCE(lh.risk,0) as localHourRisk,
COALESCE(pbcc.risk,0) as paymentBillingCountryCodeRisk,
COALESCE(pbpc.risk,0) as paymentBillingPostalCodeRisk,
COALESCE(pbst.risk,0) as paymentBillingStateRisk,
COALESCE(tcc.risk,0) as transactionCurrencyCodeRisk
from lakehousebronze.tagged_processed as t
left join Risk_AccountCountry as ac on ac.accountCountry = t.accountCountry
left join Risk_AccountPostalCode as apc on apc.accountPostalCode = t.accountPostalCode
left join Risk_AccountState as actst on actst.accountState = t.accountState
left join Risk_BrowserLanguage as bl on bl.browserLanguage = t.browserLanguage
left join Risk_IpCountryCode as ic on ic.ipCountryCode = t.ipCountryCode
left join Risk_IpPostCode as ipc on ipc.ipPostCode = t.ipPostCode
left join Risk_IpState as ips on ips.ipState = t.ipState
left join Risk_LocalHour as lh on lh.localHour = t.localHour
left join Risk_PaymentBillingCountryCode as pbcc on pbcc.paymentBillingCountryCode = t.paymentBillingCountryCode
left join Risk_PaymentBillingPostalCode as pbpc on pbpc.paymentBillingPostalCode = t.paymentBillingPostalCode
left join Risk_PaymentBillingState as pbst on pbst.paymentBillingState = t.paymentBillingState
left join Risk_TransactionCurrencyCode as tcc on tcc.transactionCurrencyCode = t.transactionCurrencyCode
) t1

LEFT OUTER JOIN
(
SELECT 
DISTINCT
CASE WHEN f1.accountid IS NULL THEN f2.accountid ELSE f1.accountid END accountid,
CASE WHEN f1.transactionID IS NULL THEN f2.transactionID ELSE f1.transactionID END transactionID,
CASE WHEN f1.TransactionDateTime IS NULL THEN f2.TransactionDateTime ELSE f1.TransactionDateTime END TransactionDateTime,
CAST(COALESCE(sumPurchaseAmount30dPerUser,0) AS DECIMAL(12,2)) sumPurchaseAmount30dPerUser,
COALESCE(sumPurchaseCount30dPerUser,0) sumPurchaseCount30dPerUser,
CAST(COALESCE(sumPurchaseAmount1dPerUser,0) AS DECIMAL(12,2)) sumPurchaseAmount1dPerUser,
COALESCE(sumPurchaseCount1dPerUser,0) sumPurchaseCount1dPerUser
FROM
fraud_final_other_txs_1 f1
FULL OUTER JOIN
fraud_final_other_txs_30 f2
ON
f1.accountid = f2.accountid AND
f1.transactionID = f2.transactionID AND
f1.TransactionDateTime = f2.TransactionDateTime
) t2
ON 
t1.accountid = t2.accountid AND
t1.transactionID = t2.transactionID AND
t1.TransactionDateTime = t2.TransactionDateTime

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [30]:
df = spark.sql("SELECT * FROM lakehousebronze.fraud_final")


StatementMeta(, 4eeecab7-b98a-437a-995d-1eb2b22708ed, 38, Finished, Available)

In [31]:
from pyspark.sql import functions as F
df.groupBy("label").count().toPandas()

StatementMeta(, 4eeecab7-b98a-437a-995d-1eb2b22708ed, 39, Finished, Available)

,label,count
0,1,1169
1,0,198326


In [32]:
df.coalesce(1).write.mode("overwrite").format("csv").save("Files/FraudData" + "frauddatafinal.csv")

StatementMeta(, 4eeecab7-b98a-437a-995d-1eb2b22708ed, 40, Finished, Available)